In [1]:
import findspark

In [2]:
# /opt/manual/spark: this is SPARK_HOME path
findspark.init("/opt/manual/spark")

In [3]:
from pyspark.sql import SparkSession, functions as F

In [4]:
spark = SparkSession.builder \
.appName("Columns and Expressions") \
.master("local[2]") \
.getOrCreate()

2022-08-27 12:26:53,006 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# Data source: https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe

In [6]:
! wget -O ~/datasets/Mall_customers \
https://github.com/erkansirin78/datasets/raw/master/Mall_Customers.csv

--2022-08-27 12:27:36--  https://github.com/erkansirin78/datasets/raw/master/Mall_Customers.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/master/Mall_Customers.csv [following]
--2022-08-27 12:27:37--  https://raw.githubusercontent.com/erkansirin78/datasets/master/Mall_Customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... failed: Connection timed out.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4365 (4.3K) [text/plain]
Saving to: ‘/home/train/datasets/Mall_customers’

100%[======================================>] 

In [7]:
! ls -l ~/datasets | grep Mall_Customers

-rw-rw-r--. 1 train train     4365 Aug 27 11:20 Mall_Customers.csv


In [8]:
spark = SparkSession \
    .builder \
    .appName("Define Custom Schema") \
    .master("local[2]") \
    .getOrCreate()

In [11]:
df = spark.read.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.option("sep", ",") \
.load("file:///home/train/datasets/Mall_Customers.csv")

In [12]:
df.limit(5).toPandas()

,CustomerID,Gender,Age,AnnualIncome,SpendingScore
0,1,Male,19,15000,39
1,2,Male,21,15000,81
2,3,Female,20,16000,6
3,4,Female,23,16000,77
4,5,Female,31,17000,40


In [14]:
df.columns

['CustomerID', 'Gender', 'Age', 'AnnualIncome', 'SpendingScore']

## Logical and math expressions on column

In [16]:
df.select(F.expr(" AnnualIncome * 10 ")).show(5)

+-------------------+
|(AnnualIncome * 10)|
+-------------------+
|             150000|
|             150000|
|             160000|
|             160000|
|             170000|
+-------------------+
only showing top 5 rows



In [18]:
# We don't want space and special characters in column name
# With alias modify auto generated column name

df.select(F.expr(" AnnualIncome * 10 ").alias("AnnualIncome_10")).show(5)

+---------------+
|AnnualIncome_10|
+---------------+
|         150000|
|         150000|
|         160000|
|         160000|
|         170000|
+---------------+
only showing top 5 rows



In [20]:
df.select( (F.col("AnnualIncome") * 12).alias("AnnualIncome_12") ).show(5)

+---------------+
|AnnualIncome_12|
+---------------+
|         180000|
|         180000|
|         192000|
|         192000|
|         204000|
+---------------+
only showing top 5 rows



# Different ways to reach a column

In [26]:
df.select(F.col("AnnualIncome")).show(3)

+------------+
|AnnualIncome|
+------------+
|       15000|
|       15000|
|       16000|
+------------+
only showing top 3 rows



In [27]:
df.select("AnnualIncome").show(3)

+------------+
|AnnualIncome|
+------------+
|       15000|
|       15000|
|       16000|
+------------+
only showing top 3 rows



In [28]:
df.select(df['AnnualIncome']).show(3)

+------------+
|AnnualIncome|
+------------+
|       15000|
|       15000|
|       16000|
+------------+
only showing top 3 rows



In [29]:
df.select(df.AnnualIncome).show(3)

+------------+
|AnnualIncome|
+------------+
|       15000|
|       15000|
|       16000|
+------------+
only showing top 3 rows



In [30]:
df.select("AnnualIncome").show(3)

+------------+
|AnnualIncome|
+------------+
|       15000|
|       15000|
|       16000|
+------------+
only showing top 3 rows



## Select multiple columns

In [31]:
df.select("CustomerID","AnnualIncome").show(3)

+----------+------------+
|CustomerID|AnnualIncome|
+----------+------------+
|         1|       15000|
|         2|       15000|
|         3|       16000|
+----------+------------+
only showing top 3 rows



In [32]:
df.select(["CustomerID","AnnualIncome"]).show(3)

+----------+------------+
|CustomerID|AnnualIncome|
+----------+------------+
|         1|       15000|
|         2|       15000|
|         3|       16000|
+----------+------------+
only showing top 3 rows



In [16]:
spark.stop()